In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys, os
import cmath as cm
import pathlib
from numpy.ma.core import shape

sys.path.insert(0, '/Users/sam/PycharmProjects/Quantum-Control/NewModule')

from NewModule.arc.PulseClass import Pulse
from PulseFunction import apply_pulse_stark, apply_pulse, change_stark_basis
from arc import *  # Import ARC (Alkali Rydberg Calculator)

In [4]:
calc = StarkMap(Calcium40())
#x%matplotlib qt

# Target state
n0 = 35
l0 = 3
j0 = 3
mj0 = 0
s0 = 0
 # Define max/min n values in basis
nmin = n0 - 1
nmax = n0 + 2


# Maximum value of l to include (l~20 gives good convergence for states with l<5)
lmax = nmax-1

# Initialise Basis States for Solver : progressOutput=True gives verbose output
calc.defineBasis(n0, l0, j0, mj0, nmin, nmax, lmax, progressOutput=True, s=s0)

Emin = 0 # Min E field (V/m)
Emax = 10  # Max E field (V/m)
N = 2  # Number of Points
E_array = np.linspace(Emin, Emax, N)  # E field array
# Generate Stark Map
calc.diagonalise(E_array, progressOutput=True,debugOutput=True, upTo = -1)
binary_matrix_diagonal = 'bi_diagonal.npy'

4.0 0.0 0.0 0.0 0.0
4.0 1.0 0.0 1.0 15157.901
4.0 1.0 1.0 1.0 15210.063
4.0 1.0 2.0 1.0 15315.943
4.0 1.0 1.0 0.0 23652.304
5.0 0.0 1.0 1.0 31539.495
5.0 0.0 0.0 0.0 33317.264
5.0 1.0 0.0 1.0 36547.688
5.0 1.0 1.0 1.0 36554.749
5.0 1.0 2.0 1.0 36575.119
5.0 1.0 1.0 0.0 36731.615
4.0 2.0 2.0 0.0 37298.287
4.0 2.0 1.0 1.0 37748.197
4.0 2.0 2.0 1.0 37751.867
4.0 2.0 3.0 1.0 37757.449
5.0 2.0 2.0 0.0 42919.053
6.0 2.0 2.0 0.0 44989.83
7.0 2.0 2.0 0.0 46200.13
8.0 2.0 2.0 0.0 46948.98
9.0 2.0 2.0 0.0 47812.39
10.0 2.0 2.0 0.0 48083.41
11.0 2.0 2.0 0.0 48290.85
12.0 2.0 2.0 0.0 48451.73
13.0 2.0 2.0 0.0 48578.32
14.0 2.0 2.0 0.0 48678.97
15.0 2.0 2.0 0.0 48760.14
16.0 2.0 2.0 0.0 48827.05
17.0 2.0 2.0 0.0 48882.37
18.0 2.0 2.0 0.0 48928.79
19.0 2.0 2.0 0.0 48968.1
20.0 2.0 2.0 0.0 49001.66
20.0 3.0 3.0 0.0 49028.943
21.0 2.0 2.0 0.0 49030.55
21.0 3.0 3.0 0.0 49054.802
22.0 2.0 2.0 0.0 49055.62
22.0 3.0 3.0 0.0 49077.203
23.0 3.0 3.0 0.0 49096.736
23.0 2.0 2.0 0.0 49096.52
24.0 3.0 3.0 0.0 49

Each stark state can be written as a sum over the atomic states $\ket{i}=\sum_k \ket{k}$. The idea is too see what is the most interesting stark state, meaning what is the stark state that has the highest coefficient associated to the level we are interested in.


In [3]:
pulse = Pulse(E_array[1],1)
psi_ini = np.zeros(len(calc.basisStates))
psi_ini[calc.indexOfCoupledState] = 1
back_to_atomic = Pulse(0,0)

ATTENTION : POUR QUE LA COMMANDE PULSE.INDEF_OF_AMPLITUDE FONCTIONNE, IL FAUT DES PULSES SUR TOUS LES VALEURS DE CHAMPS ELECTRIQUES CALCULEES.

In [4]:
print(psi_ini)
print(pulse.index_of_amplitude())
print(back_to_atomic.index_of_amplitude())

[0. 0. 0. 1. 0. 0.]
1
0


In [5]:
psi_out_stay_stark = apply_pulse_stark(psi_ini,pulse,calc)
psi_out_back_to_atomic = change_stark_basis(psi_out_stay_stark,pulse.index_of_amplitude(),back_to_atomic.index_of_amplitude(),calc=calc)

In [6]:
from PulseFunction import total_population, state_population
total = total_population(psi_out_back_to_atomic)
print(total[3])

0.9999645082809502


In [7]:
print(total[3])

0.9999645082809502


In [8]:
from PulseFunction import create_basis_change
matrix = create_basis_change(pulse,back_to_atomic,calc)
print(matrix)


[[-9.99992397e-01  3.89723811e-03 -5.94348311e-06  1.03558697e-06
  -1.31902504e-04 -2.50251444e-07]
 [ 3.89723796e-03  9.99987935e-01 -2.96750635e-03  3.59036398e-04
   5.99582293e-06 -7.81744924e-05]
 [-6.22125549e-06 -2.96747128e-03 -9.99985262e-01 -8.36258792e-05
   4.54558899e-03  1.46800796e-06]
 [ 4.20090132e-07 -3.59325672e-04 -1.09586146e-04  9.99982254e-01
  -5.94563641e-03 -1.58458498e-06]
 [ 1.31897751e-04 -5.90402484e-06 -4.54501489e-03 -5.94607574e-03
  -9.99971895e-01 -4.23004703e-04]
 [ 1.10218236e-07  7.81758212e-05 -6.86734855e-07 -9.02470721e-07
  -4.23008513e-04  9.99999907e-01]]


On check que x*x^-1 est bien l'identité.

In [9]:
inv_matrix = np.linalg.inv(matrix)
print(inv_matrix)

[[-9.99992397e-01  3.89723796e-03 -6.22125549e-06  4.20090132e-07
   1.31897751e-04  1.10218236e-07]
 [ 3.89723811e-03  9.99987935e-01 -2.96747128e-03 -3.59325672e-04
  -5.90402484e-06  7.81758212e-05]
 [-5.94348311e-06 -2.96750635e-03 -9.99985262e-01 -1.09586146e-04
  -4.54501489e-03 -6.86734855e-07]
 [ 1.03558697e-06  3.59036398e-04 -8.36258792e-05  9.99982254e-01
  -5.94607574e-03 -9.02470721e-07]
 [-1.31902504e-04  5.99582293e-06  4.54558899e-03 -5.94563641e-03
  -9.99971895e-01 -4.23008513e-04]
 [-2.50251444e-07 -7.81744924e-05  1.46800796e-06 -1.58458498e-06
  -4.23004703e-04  9.99999907e-01]]


In [10]:
a = matrix @ inv_matrix
b = inv_matrix @ matrix

On veut vérifier que apply pulse, qui fait un pulse carre, partant de la base ato va dans une base stark, evolue et retourne dans base ato. On a maintenant un apply pulse stark qui reste dans cetetat stark. Et on a moyen de creer la matrice de changement de base entre stark et atomique. DONC
apply pulse devrait etre egal a apply pulse stark + change basis

In [13]:
psi_out = apply_pulse(psi_in=psi_ini,pulse=pulse,calc=calc)
print(psi_out)


[ 7.26045674e-06+1.70031518e-05j  3.81560210e-03+2.76698355e-03j
  5.47618414e-05+7.78933916e-05j  9.99651314e-01-2.57463579e-02j
 -1.41902451e-03-3.18397713e-03j  9.43353658e-08-2.19539633e-06j]


In [14]:
x = total_population(psi_out)
print(x)

[3.41821402e-10 2.22150173e-05 9.06623973e-09 9.99965624e-01
 1.21513409e-05 4.82866421e-12]


In [15]:
psi_out_test = np.dot(matrix,psi_out)
print(psi_out_test)

[ 8.83198163e-06-5.82657922e-06j  4.17432455e-03+2.75752248e-03j
 -1.56130791e-04-9.84232864e-05j  9.99640634e-01-2.57279730e-02j
 -4.52528831e-03+3.33661024e-03j  9.06897827e-08-6.09051771e-07j]


In [16]:
y = total_population(psi_out_test)
print(y)

[1.11952925e-10 2.50289156e-05 3.40639673e-08 9.99943326e-01
 3.16112022e-05 3.79168697e-13]


On remarque que les résultats sont quelque peu différents, pourquoi ?

In [18]:
#transfo to stark basis
psi_in_stark2 = np.dot(inv_matrix,psi_ini)
print(psi_in_stark2)

[ 1.03558697e-06  3.59036398e-04 -8.36258792e-05  9.99982254e-01
 -5.94607574e-03 -9.02470721e-07]
[ 4.20090132e-07 -3.59325672e-04 -1.09586146e-04  9.99982254e-01
 -5.94563641e-03 -1.58458498e-06]


This is the initial state but transformed in the Stark basis. Now we just need to apply pulse stark

In [19]:
#evolution
a = apply_pulse_stark(psi_in_stark2,pulse,calc)
# go back to atomic
final_psi = np.dot(matrix,a)
print(final_psi)

[1.52265892e-06-5.58991650e-07j 2.59461989e-04+2.11158663e-05j
 5.42463059e-05+9.10640945e-05j 4.35475387e-01-9.00198750e-01j
 1.49886015e-03+1.03518807e-03j 2.05112839e-06-2.40522801e-06j]


In [20]:
result = total_population(final_psi)
print(result)

[2.63096184e-12 6.77664038e-08 1.12353310e-08 9.99996603e-01
 3.31819608e-06 9.99224947e-12]
